# Statistical Analysis of Basketball Performance and Winning Associations

## Initial Loading

The libraries used and datasets are initially loaded.

In [50]:
import pandas as pd

data_types = {
    "TEAM_ID":"int64",
    "team_id":"int64",
    "FGA":"int64",
    "FGM":"int64",
    "FG_PCT":"float64",
    "FG3M":"int64",
    "FG3A":"int64",
    "FG3_PCT":"float64",
    "FTM":"int64",
    "FTA":"int64",
    "FT_PCT":"float64",
    "OREB":"int64",
    "DREB":"int64",
    "REB":"int64",
    "AST":"int64",
    "TOV":"int64",
    "STL":"int64",
    "BLK":"int64",
    "BLKA":"int64",
    "PFA":"int64",
    "PFD":"int64",
    "PTS":"int64",
    "PLUS_MINUS":"int64"
}
regular_season_box_scores_1 = pd.read_csv('regular_season_box_scores_2010_2024_part_1 (1).csv',dtype=data_types,parse_dates=["game_date"])
regular_season_box_scores_2 = pd.read_csv('regular_season_box_scores_2010_2024_part_2 (1).csv',dtype=data_types,parse_dates=["game_date"])
regular_season_box_scores_3 = pd.read_csv('regular_season_box_scores_2010_2024_part_3 (1).csv',dtype=data_types,parse_dates=["game_date"])
play_off_box_scores = pd.read_csv('play_off_box_scores_2010_2024 (1).csv',dtype=data_types,parse_dates=["game_date"])
play_off_totals = pd.read_csv('play_off_totals_2010_2024 (1).csv',dtype=data_types,parse_dates=["GAME_DATE"])
regular_season_totals = pd.read_csv('regular_season_totals_2010_2024 (1).csv',dtype=data_types,parse_dates=["GAME_DATE"])

## Data Wrangling

### Data Cleaning

The box score datasets for the regular seasons are merged into one and unnecessary columns are removed while most of the remaining columns are renamed for clarity. Additionally, column types are changed to more suitable types based on the variable. The minutes column was split into minutes  seconds so that there were two numeric columns and teamCity and teamName were merged into one column as the team name combined with the city name is better known. Note that after cleaning the data, only the first 10000 rows of each dataframe are displayed.


In [51]:
#Merge all 3 dataframes of regular box scores into one

regular_season_box_scores = pd.concat([regular_season_box_scores_1,regular_season_box_scores_2,regular_season_box_scores_3])

#Remove unnecessary columns

regular_season_box_scores = regular_season_box_scores.drop(columns = ["teamSlug","jerseyNum"])
play_off_box_scores=play_off_box_scores.drop(columns=["teamSlug"])
regular_season_totals=regular_season_totals.drop(columns=["AVAILABLE_FLAG"])
play_off_totals=play_off_totals.drop(columns=["AVAILABLE_FLAG"])

#Rename columns

regular_season_box_scores = regular_season_box_scores.rename(columns={"season_year":"seasonYear","game_date":"gameDate"})
play_off_box_scores = play_off_box_scores.rename(columns={"season_year":"seasonYear","game_date":"gameDate"})
regular_season_totals = regular_season_totals.rename(columns={"SEASON_YEAR":"seasonYear","TEAM_ID":"team_ID","TEAM_ABBREVIATION":"teamAbbreviation","TEAM_NAME":"teamName","GAME_ID":"gameID",
                                                  "GAME_DATE":"gameDate","MATCHUP":"matchup","MIN":"minutes","FGA":"fieldGoalsAttempted","FGM":"fieldGoalsMade",
                                                  "FG_PCT":"fieldGoalsPercentage","FG3M":"threePointersMade","FG3A":"threePointersAttempted","FG3_PCT":"threePointersPercentage","FTM":"freeThrowsMade",
                                                  "FTA":"freeThrowsAttempted","FT_PCT":"freeThrowsPercentage","OREB":"reboundsOffensive","DREB":"reboundsDefensive","REB":"reboundsTotal",
                                                  "AST":"assists","TOV":"turnovers","STL":"steals","BLK":"blocks","BLKA":"opponentBlocks",
                                                  "PF":"foulsPersonal","PFD":"foulsPersonalDrawn","PTS":"points","PLUS_MINUS":"plusMinusPoints"})
play_off_totals = play_off_totals.rename(columns={"SEASON_YEAR":"seasonYear","TEAM_ID":"team_ID","TEAM_ABBREVIATION":"teamAbbreviation","TEAM_NAME":"teamName","GAME_ID":"gameID",
                                                        "GAME_DATE":"gameDate","MATCHUP":"matchup","MIN":"minutes","FGA":"fieldGoalsAttempted","FGM":"fieldGoalsMade",
                                                        "FG_PCT":"fieldGoalsPercentage","FG3M":"threePointersMade","FG3A":"threePointersAttempted","FG3_PCT":"threePointersPercentage",
                                                        "FTM":"freeThrowsMade","FTA":"freeThrowsAttempted","FT_PCT":"freeThrowsPercentage","OREB":"reboundsOffensive","DREB":"reboundsDefensive",
                                                        "REB":"reboundsTotal","AST":"assists","TOV":"turnovers","STL":"steals","BLK":"blocks","BLKA":"opponentBlocks",
                                                        "PF":"foulsPersonal","PFD":"foulsPersonalDrawn","PTS":"points","PLUS_MINUS":"plusMinusPoints"})


#Merge teamCity and teamName into one column

regular_season_box_scores["teamName"] = regular_season_box_scores["teamCity"] + " " + regular_season_box_scores["teamName"]

play_off_box_scores["teamName"] = play_off_box_scores["teamCity"] + " " + play_off_box_scores["teamName"]

# #Rename row values
# #Note that the Charlotte Bobcats are now the Charlotte Hornets, New Orleans Hornets are the New Orleans Pelicans, and the New Jersey Nets are the Brooklyn Nets

regular_season_box_scores["teamName"] = regular_season_box_scores["teamName"].replace(["Charlotte Bobcats","New Jersey Nets","LA Clippers", "New Orleans Hornets"],
                                                                                      ["Charlotte Hornets","Brooklyn Nets","Los Angeles Clippers","New Orleans Pelicans"])
regular_season_totals["teamName"] = regular_season_totals["teamName"].replace(["Charlotte Bobcats","New Jersey Nets","LA Clippers", "New Orleans Hornets"],
                                                                              ["Charlotte Hornets","Brooklyn Nets","Los Angeles Clippers","New Orleans Pelicans"])
play_off_box_scores["teamName"] = play_off_box_scores["teamName"].replace(["Charlotte Bobcats", "New Jersey Nets","LA Clippers","New Orleans Hornets"],
                                                                          ["Charlotte Hornets","Brooklyn Nets","Los Angeles Clippers","New Orleans Pelicans"])
play_off_totals["teamName"] = play_off_totals["teamName"].replace(["Charlotte Bobcats", "New Jersey Nets","LA Clippers","New Orleans Hornets"],
                                                                  ["Charlotte Hornets","Brooklyn Nets","Los Angeles Clippers","New Orleans Pelicans"])

#Adjust gameDate column type from chr to date

regular_season_box_scores["gameDate"] = pd.to_datetime(regular_season_box_scores["gameDate"])
regular_season_totals["gameDate"] = pd.to_datetime(regular_season_totals["gameDate"])
play_off_box_scores["gameDate"] = pd.to_datetime(play_off_box_scores["gameDate"])
play_off_totals["gameDate"] = pd.to_datetime(play_off_totals["gameDate"])

#Separate minutes and seconds

regular_season_box_scores[["minutes","seconds"]] = regular_season_box_scores["minutes"].str.split(":", expand=True)
regular_season_box_scores["minutes"] = regular_season_box_scores["minutes"].fillna(0)
regular_season_box_scores["seconds"] = regular_season_box_scores["seconds"].fillna(0)
regular_season_box_scores["minutes"] = regular_season_box_scores["minutes"].astype("int64")
regular_season_box_scores["seconds"] = regular_season_box_scores["seconds"].astype("int64")

play_off_box_scores[["minutes","seconds"]] = play_off_box_scores["minutes"].str.split(":", expand=True)
play_off_box_scores["minutes"] = play_off_box_scores["minutes"].fillna(0)
play_off_box_scores["seconds"] = play_off_box_scores["seconds"].fillna(0)
play_off_box_scores["minutes"] = play_off_box_scores["minutes"].astype("int64")
play_off_box_scores["seconds"] = play_off_box_scores["seconds"].astype("int64")


#Sort in ascending order by date

regular_season_box_scores = regular_season_box_scores.sort_values(by="gameDate")
regular_season_totals = regular_season_totals.sort_values(by="gameDate")
play_off_box_scores = play_off_box_scores.sort_values(by="gameDate")
play_off_totals = play_off_totals.sort_values(by="gameDate")

#Remove rows containing inadmissible values

regular_season_box_scores = regular_season_box_scores[(regular_season_box_scores["gameDate"]<='2024-04-14') &
                                                      ('2010-10-26'<=regular_season_box_scores["gameDate"]) &
                                                      (regular_season_box_scores["minutes"]>=0) &
                                                      (regular_season_box_scores["seconds"]>=0) &
                                                      (regular_season_box_scores["fieldGoalsAttempted"]>=0) &
                                                      (1>=regular_season_box_scores["fieldGoalsPercentage"]) &
                                                      (regular_season_box_scores["fieldGoalsPercentage"]>=0) &
                                                      (regular_season_box_scores["threePointersMade"]>=0) &
                                                      (regular_season_box_scores["threePointersAttempted"]>=0) &
                                                      (1>=regular_season_box_scores["threePointersPercentage"]) &
                                                      (regular_season_box_scores["threePointersPercentage"]>=0) &
                                                      (regular_season_box_scores["freeThrowsMade"]>=0) &
                                                      (regular_season_box_scores["freeThrowsAttempted"]>=0) &
                                                      (1>=regular_season_box_scores["freeThrowsPercentage"]) &
                                                      (regular_season_box_scores["freeThrowsPercentage"]>=0) &
                                                      (regular_season_box_scores["reboundsOffensive"]>=0) &
                                                      (regular_season_box_scores["reboundsDefensive"]>=0) &
                                                      (regular_season_box_scores["reboundsTotal"]==regular_season_box_scores["reboundsOffensive"]+regular_season_box_scores["reboundsDefensive"]) &
                                                      (regular_season_box_scores["assists"]>=0) &
                                                      (regular_season_box_scores["steals"]>=0) &
                                                      (regular_season_box_scores["blocks"]>=0) &
                                                      (regular_season_box_scores["turnovers"]>=0) &
                                                      (regular_season_box_scores["foulsPersonal"]>=0) &
                                                      (regular_season_box_scores["points"]>=0)]                                                           

play_off_box_scores = play_off_box_scores[(play_off_box_scores["gameDate"]<='2024-06-17') &
                                                      ('2011-04-16'<=play_off_box_scores["gameDate"]) &
                                                      (play_off_box_scores["minutes"]>=0) &
                                                      (play_off_box_scores["seconds"]>=0) &
                                                      (play_off_box_scores["fieldGoalsAttempted"]>=0) &
                                                      (1>=play_off_box_scores["fieldGoalsPercentage"]) &
                                                      (play_off_box_scores["fieldGoalsPercentage"]>=0) &
                                                      (play_off_box_scores["threePointersMade"]>=0) &
                                                      (play_off_box_scores["threePointersAttempted"]>=0) &
                                                      (1>=play_off_box_scores["threePointersPercentage"]) &
                                                      (play_off_box_scores["threePointersPercentage"]>=0) &
                                                      (play_off_box_scores["freeThrowsMade"]>=0) &
                                                      (play_off_box_scores["freeThrowsAttempted"]>=0) &
                                                      (1>=play_off_box_scores["freeThrowsPercentage"]) &
                                                      (play_off_box_scores["freeThrowsPercentage"]>=0) &
                                                      (play_off_box_scores["reboundsOffensive"]>=0) &
                                                      (play_off_box_scores["reboundsDefensive"]>=0) &
                                                      (play_off_box_scores["reboundsTotal"]==play_off_box_scores["reboundsOffensive"]+play_off_box_scores["reboundsDefensive"]) &
                                                      (play_off_box_scores["assists"]>=0) &
                                                      (play_off_box_scores["steals"]>=0) &
                                                      (play_off_box_scores["blocks"]>=0) &
                                                      (play_off_box_scores["turnovers"]>=0) &
                                                      (play_off_box_scores["foulsPersonal"]>=0) &
                                                      (play_off_box_scores["points"]>=0)]      

regular_season_totals = regular_season_totals[(regular_season_totals["gameDate"]<='2024-04-14') &
                                                      ('2010-10-26'<=regular_season_totals["gameDate"]) &
                                                      (regular_season_totals["minutes"]>=0) &
                                                      (regular_season_totals["fieldGoalsAttempted"]>=0) &
                                                      (1>=regular_season_totals["fieldGoalsPercentage"]) &
                                                      (regular_season_totals["fieldGoalsPercentage"]>=0) &
                                                      (regular_season_totals["threePointersMade"]>=0) &
                                                      (regular_season_totals["threePointersAttempted"]>=0) &
                                                      (1>=regular_season_totals["threePointersPercentage"]) &
                                                      (regular_season_totals["threePointersPercentage"]>=0) &
                                                      (regular_season_totals["freeThrowsMade"]>=0) &
                                                      (regular_season_totals["freeThrowsAttempted"]>=0) &
                                                      (1>=regular_season_totals["freeThrowsPercentage"]) &
                                                      (regular_season_totals["freeThrowsPercentage"]>=0) &
                                                      (regular_season_totals["reboundsOffensive"]>=0) &
                                                      (regular_season_totals["reboundsDefensive"]>=0) &
                                                      (regular_season_totals["reboundsTotal"]==regular_season_totals["reboundsOffensive"]+regular_season_totals["reboundsDefensive"]) &
                                                      (regular_season_totals["assists"]>=0) &
                                                      (regular_season_totals["steals"]>=0) &
                                                      (regular_season_totals["blocks"]>=0) &
                                                      (regular_season_totals["turnovers"]>=0) &
                                                      (regular_season_totals["foulsPersonal"]>=0) &
                                                      (regular_season_totals["foulsPersonalDrawn"]>=0) &
                                                      (regular_season_totals["points"]>=0)] 

play_off_totals = play_off_totals[(play_off_totals["gameDate"]<='2024-06-17') &
                                                      ('2011-04-11'<=play_off_totals["gameDate"]) &
                                                      (play_off_totals["minutes"]>=0) &
                                                      (play_off_totals["fieldGoalsAttempted"]>=0) &
                                                      (1>=play_off_totals["fieldGoalsPercentage"]) &
                                                      (play_off_totals["fieldGoalsPercentage"]>=0) &
                                                      (play_off_totals["threePointersMade"]>=0) &
                                                      (play_off_totals["threePointersAttempted"]>=0) &
                                                      (1>=play_off_totals["threePointersPercentage"]) &
                                                      (play_off_totals["threePointersPercentage"]>=0) &
                                                      (play_off_totals["freeThrowsMade"]>=0) &
                                                      (play_off_totals["freeThrowsAttempted"]>=0) &
                                                      (1>=play_off_totals["freeThrowsPercentage"]) &
                                                      (play_off_totals["freeThrowsPercentage"]>=0) &
                                                      (play_off_totals["reboundsOffensive"]>=0) &
                                                      (play_off_totals["reboundsDefensive"]>=0) &
                                                      (play_off_totals["reboundsTotal"]==play_off_totals["reboundsOffensive"]+play_off_totals["reboundsDefensive"]) &
                                                      (play_off_totals["assists"]>=0) &
                                                      (play_off_totals["steals"]>=0) &
                                                      (play_off_totals["blocks"]>=0) &
                                                      (play_off_totals["turnovers"]>=0) &
                                                      (play_off_totals["foulsPersonal"]>=0) &
                                                      (play_off_totals["foulsPersonalDrawn"]>=0) &
                                                      (play_off_totals["points"]>=0)] 

In [52]:
regular_season_box_scores

,seasonYear,gameDate,gameId,matchup,teamId,teamCity,teamName,teamTricode,personId,personName,...,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,seconds
71276,2010-11,2010-10-26,21000001,MIA @ BOS,1610612748,Miami,Miami Heat,MIA,2544,LeBron James,...,4,4,3,1,2,8,1,31,1,39
69652,2010-11,2010-10-26,21000001,MIA @ BOS,1610612748,Miami,Miami Heat,MIA,436,Juwan Howard,...,0,0,0,0,0,0,0,0,0,0
28447,2010-11,2010-10-26,21000001,BOS vs. MIA,1610612738,Boston,Boston Celtics,BOS,708,Kevin Garnett,...,8,10,3,3,0,7,5,10,7,13
48125,2010-11,2010-10-26,21000003,LAL vs. HOU,1610612747,Los Angeles,Los Angeles Lakers,LAL,202365,Devin Ebanks,...,0,0,0,0,0,0,0,0,0,0
60750,2010-11,2010-10-26,21000002,POR vs. PHX,1610612757,Portland,Portland Trail Blazers,POR,202337,Luke Babbitt,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43467,2023-24,2024-04-14,22301195,LAL @ NOP,1610612747,Los Angeles,Los Angeles Lakers,LAL,2544,LeBron James,...,9,11,17,5,1,4,0,28,19,41
140032,2023-24,2024-04-14,22301190,NYK vs. CHI,1610612752,New York,New York Knicks,NYK,1630173,Precious Achiuwa,...,4,5,2,0,1,2,3,4,-2,30
26593,2023-24,2024-04-14,22301191,ORL vs. MIL,1610612753,Orlando,Orlando Magic,ORL,1630175,Cole Anthony,...,3,5,4,2,0,1,3,13,13,3
137517,2023-24,2024-04-14,22301190,NYK vs. CHI,1610612752,New York,New York Knicks,NYK,1628404,Josh Hart,...,7,10,4,1,1,3,5,12,-4,8


In [53]:
play_off_box_scores

,seasonYear,gameDate,gameId,teamId,teamCity,teamName,teamTricode,personId,personName,position,...,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints,seconds
28167,2010-11,2011-04-16,41000161,1610612757,Portland,Portland Trail Blazers,POR,201164,Rudy Fernandez,NaN,...,0,2,1,1,0,0,2,6,0,20
7013,2010-11,2011-04-16,41000101,1610612741,Chicago,Chicago Bulls,CHI,200758,Ronnie Brewer,NaN,...,2,3,1,2,0,0,0,2,-2,13
623,2010-11,2011-04-16,41000111,1610612748,Miami,Miami Heat,MIA,2548,Dwyane Wade,G,...,6,7,5,0,1,2,5,17,16,23
101,2010-11,2011-04-16,41000111,1610612748,Miami,Miami Heat,MIA,980,Zydrunas Ilgauskas,C,...,4,6,1,0,0,0,1,8,-4,16
12474,2010-11,2011-04-16,41000101,1610612754,Indiana,Indiana Pacers,IND,201954,Darren Collison,G,...,5,6,9,2,1,1,2,17,-3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29556,2023-24,2024-06-17,42300405,1610612742,Dallas,Dallas Mavericks,DAL,202693,Markieff Morris,NaN,...,0,0,0,0,0,0,0,0,0,0
29004,2023-24,2024-06-17,42300405,1610612738,Boston,Boston Celtics,BOS,1628369,Jayson Tatum,F,...,7,8,11,2,0,2,2,31,18,57
29118,2023-24,2024-06-17,42300405,1610612738,Boston,Boston Celtics,BOS,1629052,Oshae Brissett,NaN,...,0,0,0,0,0,0,0,0,0,39
29433,2023-24,2024-06-17,42300405,1610612742,Dallas,Dallas Mavericks,DAL,1629029,Luka Doncic,G,...,12,12,5,3,0,7,3,28,-19,23


In [38]:
regular_season_totals

,seasonYear,team_ID,teamAbbreviation,teamName,gameID,gameDate,matchup,WL,minutes,fieldGoalsMade,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
6649,2010-11,1610612747,LAL,Los Angeles Lakers,21000003,2010-10-26,LAL vs. HOU,W,48.0,40,...,756,1197,567,174,1269,1877,1843,322,337,1116
9213,2010-11,1610612738,BOS,Boston Celtics,21000001,2010-10-26,BOS vs. MIA,W,48.0,32,...,1050,502,2257,1459,1269,1586,767,1023,2031,634
4394,2010-11,1610612757,POR,Portland Trail Blazers,21000002,2010-10-26,POR vs. PHX,W,48.0,43,...,330,65,1335,174,2024,813,1439,1474,690,305
9283,2010-11,1610612745,HOU,Houston Rockets,21000003,2010-10-26,HOU @ LAL,L,48.0,38,...,85,502,2326,1459,368,813,2012,450,437,1288
6727,2010-11,1610612756,PHX,Phoenix Suns,21000002,2010-10-26,PHX @ POR,L,48.0,36,...,2373,2190,2145,2262,1269,174,767,806,1763,2112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9756,2023-24,1610612740,NOP,New Orleans Pelicans,22301195,2024-04-14,NOP vs. LAL,L,48.0,41,...,1785,1241,2204,839,2322,382,737,2212,1635,2096
32129,2023-24,1610612757,POR,Portland Trail Blazers,22301200,2024-04-14,POR @ SAC,L,48.0,31,...,140,2359,2077,205,2080,1459,1208,1020,2449,2437
19548,2023-24,1610612752,NYK,New York Knicks,22301190,2024-04-14,NYK vs. CHI,W,53.0,46,...,181,1031,2357,1161,690,382,737,596,763,1185
23458,2023-24,1610612745,HOU,Houston Rockets,22301199,2024-04-14,HOU @ LAC,W,48.0,50,...,22,423,2077,1161,267,2012,143,2080,1044,547


In [39]:
play_off_totals

,seasonYear,team_ID,teamAbbreviation,teamName,gameID,gameDate,matchup,WL,minutes,fieldGoalsMade,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
94,2010-11,1610612741,CHI,Chicago Bulls,41000101,2011-04-16,CHI vs. IND,W,48.0,36,...,18,80,107,101,8,94,68,28,21,58
72,2010-11,1610612755,PHI,Philadelphia 76ers,41000111,2011-04-16,PHI @ MIA,L,48.0,35,...,84,23,9,35,146,66,124,142,103,117
75,2010-11,1610612757,POR,Portland Trail Blazers,41000161,2011-04-16,POR @ DAL,L,48.0,35,...,77,23,87,67,122,109,136,142,150,117
5,2010-11,1610612737,ATL,Atlanta Hawks,41000131,2011-04-16,ATL @ ORL,W,48.0,38,...,159,62,16,35,158,18,152,40,24,28
151,2010-11,1610612753,ORL,Orlando Magic,41000131,2011-04-16,ORL vs. ATL,L,48.0,34,...,77,123,141,153,122,1,108,7,80,131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,2023-24,1610612738,BOS,Boston Celtics,42300403,2024-06-12,BOS @ DAL,W,48.0,38,...,128,35,19,131,39,4,79,104,75,53
2201,2023-24,1610612738,BOS,Boston Celtics,42300404,2024-06-14,BOS @ DAL,L,48.0,29,...,153,136,102,158,65,12,79,104,158,162
2200,2023-24,1610612742,DAL,Dallas Mavericks,42300404,2024-06-14,DAL vs. BOS,W,48.0,46,...,7,103,19,53,132,81,44,73,13,2
2199,2023-24,1610612738,BOS,Boston Celtics,42300405,2024-06-17,BOS vs. DAL,W,48.0,38,...,13,49,19,18,132,57,17,56,75,23


## Summary Statistics

By player, per season

In [54]:
regular_season_player_summary_stats = regular_season_box_scores[(~regular_season_box_scores["comment"].str.contains("DND|DNP|NWT",na=False,case=False))]
regular_season_player_summary_stats = regular_season_player_summary_stats.groupby(["seasonYear","personName","teamName"]).agg(totalMinutesPlayed = ("minutes","sum"),
                                                                                                                            avgMinutesPlayed=("minutes","mean"),
                                                                                                                            totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                                            avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                                            sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                                            totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                                            avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                                            avgFieldGoalPercentage=("fieldGoalsPercentage","mean"),
                                                                                                                            totalThreePointersMade=("threePointersMade","sum"),
                                                                                                                            avgThreePointersMade=("threePointersMade","mean"),
                                                                                                                            sdThreePointersMade=("threePointersMade","std"),
                                                                                                                            totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                                            avgThreePointersAttempted=("threePointersAttempted","mean"),
                                                                                                                            avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                                            totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                                            avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                                            totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                                            avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                                            sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                                            avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                                            totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                                            avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                                            sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                                            totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                                            avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                                            sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                                            totalRebounds=("reboundsTotal","sum"),
                                                                                                                            avgRebounds=("reboundsTotal","mean"),
                                                                                                                            sdRebounds=("reboundsTotal","std"),
                                                                                                                            totalAssists=("assists","std"),
                                                                                                                            avgAssists=("assists","mean"),
                                                                                                                            sdAssists=("assists","std"),
                                                                                                                            totalSteals=("steals","sum"),
                                                                                                                            avgSteals=("steals","mean"),
                                                                                                                            sdSteals=("steals","std"),
                                                                                                                            totalBlocks=("blocks","sum"),
                                                                                                                            avgBlocks=("blocks","mean"),
                                                                                                                            sdBlocks=("blocks","std"),
                                                                                                                            totalTurnovers=("turnovers","sum"),
                                                                                                                            avgTurnovers=("turnovers","mean"),
                                                                                                                            sdTurnovers=("turnovers","std"),
                                                                                                                            totalFouls=("foulsPersonal","sum"),
                                                                                                                            avgFouls=("foulsPersonal","mean"),
                                                                                                                            sdFouls=("foulsPersonal","std"),
                                                                                                                            totalPoints=("points","sum"),
                                                                                                                            avgPoints=("points","mean"),
                                                                                                                            sdPoints=("points","std"),
                                                                                                                            avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                                            totalPlusMinusPoints=("plusMinusPoints","sum"))

regular_season_player_summary_stats["totalFieldGoalsPercentage"] = regular_season_player_summary_stats["totalFieldGoalsMade"]/regular_season_player_summary_stats["totalFieldGoalsAttempted"]
regular_season_player_summary_stats["totalThreePointersPercentage"] = regular_season_player_summary_stats["totalThreePointersMade"]/regular_season_player_summary_stats["totalThreePointersAttempted"]
regular_season_player_summary_stats["totalFreeThrowsPercentage"] = regular_season_player_summary_stats["totalFreeThrowsMade"]/regular_season_player_summary_stats["totalFreeThrowsAttempted"]

play_off_player_summary_stats = play_off_box_scores[(~play_off_box_scores["comment"].str.contains("DND|DNP|NWT",na=False,case=False))]
play_off_player_summary_stats = play_off_player_summary_stats.groupby(["seasonYear","personName","teamName"]).agg(totalMinutesPlayed = ("minutes","sum"),
                                                                                                                            avgMinutesPlayed=("minutes","mean"),
                                                                                                                            totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                                            avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                                            sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                                            totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                                            avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                                            avgFieldGoalPercentage=("fieldGoalsPercentage","mean"),
                                                                                                                            totalThreePointersMade=("threePointersMade","sum"),
                                                                                                                            avgThreePointersMade=("threePointersMade","mean"),
                                                                                                                            sdThreePointersMade=("threePointersMade","std"),
                                                                                                                            totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                                            avgThreePointersAttempted=("threePointersAttempted","mean"),
                                                                                                                            avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                                            totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                                            avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                                            totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                                            avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                                            sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                                            avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                                            totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                                            avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                                            sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                                            totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                                            avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                                            sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                                            totalRebounds=("reboundsTotal","sum"),
                                                                                                                            avgRebounds=("reboundsTotal","mean"),
                                                                                                                            sdRebounds=("reboundsTotal","std"),
                                                                                                                            totalAssists=("assists","std"),
                                                                                                                            avgAssists=("assists","mean"),
                                                                                                                            sdAssists=("assists","std"),
                                                                                                                            totalSteals=("steals","sum"),
                                                                                                                            avgSteals=("steals","mean"),
                                                                                                                            sdSteals=("steals","std"),
                                                                                                                            totalBlocks=("blocks","sum"),
                                                                                                                            avgBlocks=("blocks","mean"),
                                                                                                                            sdBlocks=("blocks","std"),
                                                                                                                            totalTurnovers=("turnovers","sum"),
                                                                                                                            avgTurnovers=("turnovers","mean"),
                                                                                                                            sdTurnovers=("turnovers","std"),
                                                                                                                            totalFouls=("foulsPersonal","sum"),
                                                                                                                            avgFouls=("foulsPersonal","mean"),
                                                                                                                            sdFouls=("foulsPersonal","std"),
                                                                                                                            totalPoints=("points","sum"),
                                                                                                                            avgPoints=("points","mean"),
                                                                                                                            sdPoints=("points","std"),
                                                                                                                            avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                                            totalPlusMinusPoints=("plusMinusPoints","sum"))

play_off_player_summary_stats["totalFieldGoalsPercentage"] = play_off_player_summary_stats["totalFieldGoalsMade"]/play_off_player_summary_stats["totalFieldGoalsAttempted"]
play_off_player_summary_stats["totalThreePointersPercentage"] = play_off_player_summary_stats["totalThreePointersMade"]/play_off_player_summary_stats["totalThreePointersAttempted"]
play_off_player_summary_stats["totalFreeThrowsPercentage"] = play_off_player_summary_stats["totalFreeThrowsMade"]/play_off_player_summary_stats["totalFreeThrowsAttempted"]

In [41]:
regular_season_player_summary_stats

totalMinutesPlayed  \
seasonYear personName      teamName                                    
2010-11    AJ Price        Indiana Pacers                        769   
           Aaron Brooks    Houston Rockets                       796   
                           Phoenix Suns                          462   
           Aaron Gray      New Orleans Pelicans                  514   
           Acie Law        Golden State Warriors                 617   
...                                                              ...   
2023-24    Zach LaVine     Chicago Bulls                         859   
           Zavier Simpson  Memphis Grizzlies                     157   
           Zeke Nnaji      Denver Nuggets                        548   
           Ziaire Williams Memphis Grizzlies                    1011   
           Zion Williamson New Orleans Pelicans                 2166   

                                                  avgMinutesPlayed  \
seasonYear personName      teamName                                  
2010-11    AJ Price        Indiana Pacers                15.380000   
           Aaron Brooks    Houston Rockets               23.411765   
                           Phoenix Suns                  18.480000   
           Aaron Gray      New Orleans Pelicans          12.536585   
           Acie Law        Golden State Warriors         15.425000   
...                                                            ...   
2023-24    Zach LaVine     Chicago Bulls                 34.360000   
           Zavier Simpson  Memphis Grizzlies             22.428571   
           Zeke Nnaji      Denver Nuggets                 9.448276   
           Ziaire Williams Memphis Grizzlies             19.823529   
           Zion Williamson New Orleans Pelicans          30.942857   

                                                  totalFieldGoalsMade  \
seasonYear personName      teamName                                     
2010-11    AJ Price        Indiana Pacers                         114   
           Aaron Brooks    Houston Rockets                        134   
                           Phoenix Suns                            86   
           Aaron Gray      New Orleans Pelicans                    56   
           Acie Law        Golden State Warriors                   78   
...                                                               ...   
2023-24    Zach LaVine     Chicago Bulls                          170   
           Zavier Simpson  Memphis Grizzlies                       17   
           Zeke Nnaji      Denver Nuggets                          69   
           Ziaire Williams Memphis Grizzlies                      150   
           Zion Williamson New Orleans Pelicans                   624   

                                                  avgFieldGoalsMade  \
seasonYear personName      teamName                                   
2010-11    AJ Price        Indiana Pacers                  2.280000   
           Aaron Brooks    Houston Rockets                 3.941176   
                           Phoenix Suns                    3.440000   
           Aaron Gray      New Orleans Pelicans            1.365854   
           Acie Law        Golden State Warriors           1.950000   
...                                                             ...   
2023-24    Zach LaVine     Chicago Bulls                   6.800000   
           Zavier Simpson  Memphis Grizzlies               2.428571   
           Zeke Nnaji      Denver Nuggets                  1.189655   
           Ziaire Williams Memphis Grizzlies               2.941176   
           Zion Williamson New Orleans Pelicans            8.914286   

                                                  sdFieldGoalsMade  \
seasonYear personName      teamName                                  
2010-11    AJ Price        Indiana Pacers                 1.512398   
           Aaron Brooks    Houston Rockets                2.282226   
                           Phoenix Suns         

In [42]:
play_off_player_summary_stats

totalMinutesPlayed  \
seasonYear personName         teamName                                     
2010-11    AJ Price           Indiana Pacers                          78   
           Aaron Gray         New Orleans Pelicans                    85   
           Al Harrington      Denver Nuggets                          68   
           Al Horford         Atlanta Hawks                          461   
           Amar'e Stoudemire  New York Knicks                        131   
...                                                                  ...   
2023-24    Tyrese Maxey       Philadelphia 76ers                     351   
           Wendell Carter Jr. Orlando Magic                          215   
           Wendell Moore Jr.  Minnesota Timberwolves                  19   
           Xavier Tillman     Boston Celtics                          66   
           Zeke Nnaji         Denver Nuggets                          13   

                                                      avgMinutesPlayed  \
seasonYear personName         teamName                                   
2010-11    AJ Price           Indiana Pacers                 15.600000   
           Aaron Gray         New Orleans Pelicans           14.166667   
           Al Harrington      Denver Nuggets                 13.600000   
           Al Horford         Atlanta Hawks                  38.416667   
           Amar'e Stoudemire  New York Knicks                32.750000   
...                                                                ...   
2023-24    Tyrese Maxey       Philadelphia 76ers             43.875000   
           Wendell Carter Jr. Orlando Magic                  23.888889   
           Wendell Moore Jr.  Minnesota Timberwolves          2.714286   
           Xavier Tillman     Boston Celtics                  8.250000   
           Zeke Nnaji         Denver Nuggets                  4.333333   

                                                      totalFieldGoalsMade  \
seasonYear personName         teamName                                      
2010-11    AJ Price           Indiana Pacers                           13   
           Aaron Gray         New Orleans Pelicans                      9   
           Al Harrington      Denver Nuggets                           10   
           Al Horford         Atlanta Hawks                            58   
           Amar'e Stoudemire  New York Knicks                          21   
...                                                                   ...   
2023-24    Tyrese Maxey       Philadelphia 76ers                       92   
           Wendell Carter Jr. Orlando Magic                            21   
           Wendell Moore Jr.  Minnesota Timberwolves                    3   
           Xavier Tillman     Boston Celtics                            5   
           Zeke Nnaji         Denver Nuggets                            2   

                                                      avgFieldGoalsMade  \
seasonYear personName         teamName                                    
2010-11    AJ Price           Indiana Pacers                   2.600000   
           Aaron Gray         New Orleans Pelicans             1.500000   
           Al Harrington      Denver Nuggets                   2.000000   
           Al Horford         Atlanta Hawks                    4.833333   
           Amar'e Stoudemire  New York Knicks                  5.250000   
...                                                                 ...   
2023-24    Tyrese Maxey       Philadelphia 76ers              11.500000   
           Wendell Carter Jr. Orlando Magic                    2.333333   
           Wendell Moore Jr.  Minnesota Timberwolves           0.428571   
           Xavier Tillman     Boston Celtics                   0.625000   
           Zeke Nnaji         Denver Nuggets                   0.666667   

                                                      sdFieldGoalsMade  \
seasonYear personName         teamName            

By team, per season

In [55]:
#Add total minutes all players played on a team per season

regular_season_player_stats = regular_season_player_summary_stats.groupby(["teamName","seasonYear"]).agg(totalMinutesPlayersPlayed = ("totalMinutesPlayed","sum"))

play_off_player_stats = play_off_player_summary_stats.groupby(["teamName","seasonYear"]).agg(totalMinutesPlayersPlayed = ("totalMinutesPlayed","sum"))

regular_season_team_summary_stats = regular_season_totals.groupby(["teamName","seasonYear"]).agg(totalWins=("WL",lambda total: total.str.count("W").sum()),
                                                                                               totalLosses=("WL",lambda total: total.str.count("L").sum()),
                                                                                               avgWins=("WL",lambda total: total.str.count("W").mean()),
                                                                                               avgLosses=("WL",lambda total: total.str.count("L").mean()),
                                                                                               totalMinutesPlayed=("minutes","sum"),
                                                                                               avgMinutesPlayed=("minutes","mean"),
                                                                                                totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                avgFieldGoalsPercentage=("fieldGoalsPercentage","mean"),
                                                                                                sdFieldGoalsPercentage=("fieldGoalsPercentage","std"),
                                                                                                totalThreePointersMade=("threePointersMade","sum"),
                                                                                                avgThreePointersMade=("threePointersMade","mean"),
                                                                                                sdThreePointersMade=("threePointersMade","std"),
                                                                                                totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                sdThreePointersPercentage=("threePointersPercentage","std"),
                                                                                                totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                sdFreeThrowsPercentage=("freeThrowsPercentage","std"),
                                                                                                totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                totalRebounds=("reboundsTotal","sum"),
                                                                                                avgRebounds=("reboundsTotal","mean"),
                                                                                                sdRebounds=("reboundsTotal","std"),
                                                                                                totalAssists=("assists","sum"),
                                                                                                avgAssists=("assists","mean"),
                                                                                                sdAssists=("assists","std"),
                                                                                                totalTurnovers=("turnovers","sum"),
                                                                                                avgTurnovers=("turnovers","mean"),
                                                                                                sdTurnovers=("turnovers","std"),
                                                                                                totalSteals=("steals","sum"),
                                                                                                avgSteals=("steals","mean"),
                                                                                                sdSteals=("steals","std"),
                                                                                                totalBlocks=("blocks","sum"),
                                                                                                avgBlocks=("blocks","mean"),
                                                                                                sdBlocks=("blocks","std"),
                                                                                                totalOpponentBlocks=("opponentBlocks","sum"),
                                                                                                avgOpponentBlocks=("opponentBlocks","mean"),
                                                                                                sdOpponentBlocks=("opponentBlocks","std"),
                                                                                                totalFouls=("foulsPersonal","sum"),
                                                                                                avgFouls=("foulsPersonal","mean"),
                                                                                                sdFouls=("foulsPersonal","std"),
                                                                                                totalDrawnFouls=("foulsPersonalDrawn","sum"),
                                                                                                avgDrawnFouls=("foulsPersonalDrawn","mean"),
                                                                                                sdDrawnFouls=("foulsPersonalDrawn","std"),
                                                                                                totalPoints=("points","sum"),
                                                                                                avgPoints=("points","mean"),
                                                                                                sdPoints=("points","std"),
                                                                                                avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                totalPlusMinusPoints=("plusMinusPoints","sum")).reset_index()

regular_season_team_summary_stats["totalFieldGoalsPercentage"] = regular_season_team_summary_stats["totalFieldGoalsMade"]/regular_season_team_summary_stats["totalFieldGoalsAttempted"]
regular_season_team_summary_stats["totalThreePointersPercentage"] = regular_season_team_summary_stats["totalThreePointersMade"]/regular_season_team_summary_stats["totalThreePointersAttempted"]
regular_season_team_summary_stats["totalFreeThrowsPercentage"] = regular_season_team_summary_stats["totalFreeThrowsMade"]/regular_season_team_summary_stats["totalFreeThrowsAttempted"]
regular_season_team_summary_stats = pd.merge(regular_season_team_summary_stats,regular_season_player_stats,how="left",on=["seasonYear","teamName"])

play_off_team_summary_stats = play_off_totals.groupby(["teamName","seasonYear"]).agg(totalWins=("WL",lambda total: total.str.count("W").sum()),
                                                                                               totalLosses=("WL",lambda total: total.str.count("L").sum()),
                                                                                               avgWins=("WL",lambda total: total.str.count("W").mean()),
                                                                                               avgLosses=("WL",lambda total: total.str.count("L").mean()),
                                                                                               totalMinutesPlayed=("minutes","sum"),
                                                                                               avgMinutesPlayed=("minutes","mean"),
                                                                                                totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                avgFieldGoalsPercentage=("fieldGoalsPercentage","mean"),
                                                                                                sdFieldGoalsPercentage=("fieldGoalsPercentage","std"),
                                                                                                totalThreePointersMade=("threePointersMade","sum"),
                                                                                                avgThreePointersMade=("threePointersMade","mean"),
                                                                                                sdThreePointersMade=("threePointersMade","std"),
                                                                                                totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                sdThreePointersPercentage=("threePointersPercentage","std"),
                                                                                                totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                sdFreeThrowsPercentage=("freeThrowsPercentage","std"),
                                                                                                totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                totalRebounds=("reboundsTotal","sum"),
                                                                                                avgRebounds=("reboundsTotal","mean"),
                                                                                                sdRebounds=("reboundsTotal","std"),
                                                                                                totalAssists=("assists","sum"),
                                                                                                avgAssists=("assists","mean"),
                                                                                                sdAssists=("assists","std"),
                                                                                                totalTurnovers=("turnovers","sum"),
                                                                                                avgTurnovers=("turnovers","mean"),
                                                                                                sdTurnovers=("turnovers","std"),
                                                                                                totalSteals=("steals","sum"),
                                                                                                avgSteals=("steals","mean"),
                                                                                                sdSteals=("steals","std"),
                                                                                                totalBlocks=("blocks","sum"),
                                                                                                avgBlocks=("blocks","mean"),
                                                                                                sdBlocks=("blocks","std"),
                                                                                                totalOpponentBlocks=("opponentBlocks","sum"),
                                                                                                avgOpponentBlocks=("opponentBlocks","mean"),
                                                                                                sdOpponentBlocks=("opponentBlocks","std"),
                                                                                                totalFouls=("foulsPersonal","sum"),
                                                                                                avgFouls=("foulsPersonal","mean"),
                                                                                                sdFouls=("foulsPersonal","std"),
                                                                                                totalDrawnFouls=("foulsPersonalDrawn","sum"),
                                                                                                avgDrawnFouls=("foulsPersonalDrawn","mean"),
                                                                                                sdDrawnFouls=("foulsPersonalDrawn","std"),
                                                                                                totalPoints=("points","sum"),
                                                                                                avgPoints=("points","mean"),
                                                                                                sdPoints=("points","std"),
                                                                                                avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                totalPlusMinusPoints=("plusMinusPoints","sum")).reset_index()

play_off_team_summary_stats["totalFieldGoalsPercentage"] = play_off_team_summary_stats["totalFieldGoalsMade"]/play_off_team_summary_stats["totalFieldGoalsAttempted"]
play_off_team_summary_stats["totalThreePointersPercentage"] = play_off_team_summary_stats["totalThreePointersMade"]/play_off_team_summary_stats["totalThreePointersAttempted"]
play_off_team_summary_stats["totalFreeThrowsPercentage"] = play_off_team_summary_stats["totalFreeThrowsMade"]/play_off_team_summary_stats["totalFreeThrowsAttempted"]
play_off_team_summary_stats = pd.merge(play_off_team_summary_stats,play_off_player_stats,how="left",on=["seasonYear","teamName"])

In [44]:
regular_season_team_summary_stats.to_csv("regular_season_team_summary_stats.csv",index=False)

By player, all time.

In [56]:
regular_season_player_all_time_summary_stats = regular_season_box_scores[(~regular_season_box_scores["comment"].str.contains("DND|DNP|NWT",na=False,case=False))]
regular_season_player_all_time_summary_stats = regular_season_player_all_time_summary_stats.groupby(["personName","teamName"]).agg(totalMinutesPlayed = ("minutes","sum"),
                                                                                                                            avgMinutesPlayed=("minutes","mean"),
                                                                                                                            totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                                            avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                                            sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                                            totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                                            avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                                            avgFieldGoalPercentage=("fieldGoalsPercentage","mean"),
                                                                                                                            totalThreePointersMade=("threePointersMade","sum"),
                                                                                                                            avgThreePointersMade=("threePointersMade","mean"),
                                                                                                                            sdThreePointersMade=("threePointersMade","std"),
                                                                                                                            totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                                            avgThreePointersAttempted=("threePointersAttempted","mean"),
                                                                                                                            avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                                            totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                                            avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                                            totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                                            avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                                            sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                                            avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                                            totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                                            avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                                            sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                                            totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                                            avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                                            sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                                            totalRebounds=("reboundsTotal","sum"),
                                                                                                                            avgRebounds=("reboundsTotal","mean"),
                                                                                                                            sdRebounds=("reboundsTotal","std"),
                                                                                                                            totalAssists=("assists","sum"),
                                                                                                                            avgAssists=("assists","mean"),
                                                                                                                            sdAssists=("assists","std"),
                                                                                                                            totalSteals=("steals","sum"),
                                                                                                                            avgSteals=("steals","mean"),
                                                                                                                            sdSteals=("steals","std"),
                                                                                                                            totalBlocks=("blocks","sum"),
                                                                                                                            avgBlocks=("blocks","mean"),
                                                                                                                            sdBlocks=("blocks","std"),
                                                                                                                            totalTurnovers=("turnovers","sum"),
                                                                                                                            avgTurnovers=("turnovers","mean"),
                                                                                                                            sdTurnovers=("turnovers","std"),
                                                                                                                            totalFouls=("foulsPersonal","sum"),
                                                                                                                            avgFouls=("foulsPersonal","mean"),
                                                                                                                            sdFouls=("foulsPersonal","std"),
                                                                                                                            totalPoints=("points","sum"),
                                                                                                                            avgPoints=("points","mean"),
                                                                                                                            sdPoints=("points","std"),
                                                                                                                            avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                                            totalPlusMinusPoints=("plusMinusPoints","sum"))

regular_season_player_all_time_summary_stats["totalFieldGoalsPercentage"] \
    = regular_season_player_all_time_summary_stats["totalFieldGoalsMade"]/regular_season_player_all_time_summary_stats["totalFieldGoalsAttempted"]
regular_season_player_all_time_summary_stats["totalThreePointersPercentage"] \
    = regular_season_player_all_time_summary_stats["totalThreePointersMade"]/regular_season_player_all_time_summary_stats["totalThreePointersAttempted"]
regular_season_player_all_time_summary_stats["totalFreeThrowsPercentage"] \
    = regular_season_player_all_time_summary_stats["totalFreeThrowsMade"]/regular_season_player_all_time_summary_stats["totalFreeThrowsAttempted"]

play_off_player_all_time_summary_stats = play_off_box_scores[(~play_off_box_scores["comment"].str.contains("DND|DNP|NWT",na=False,case=False))]
play_off_player_all_time_summary_stats = play_off_player_all_time_summary_stats.groupby(["personName","teamName"]).agg(totalMinutesPlayed = ("minutes","sum"),
                                                                                                                            avgMinutesPlayed=("minutes","mean"),
                                                                                                                            totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                                            avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                                            sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                                            totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                                            avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                                            avgFieldGoalPercentage=("fieldGoalsPercentage","mean"),
                                                                                                                            totalThreePointersMade=("threePointersMade","sum"),
                                                                                                                            avgThreePointersMade=("threePointersMade","mean"),
                                                                                                                            sdThreePointersMade=("threePointersMade","std"),
                                                                                                                            totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                                            avgThreePointersAttempted=("threePointersAttempted","mean"),
                                                                                                                            avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                                            totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                                            avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                                            totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                                            avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                                            sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                                            avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                                            totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                                            avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                                            sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                                            totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                                            avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                                            sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                                            totalRebounds=("reboundsTotal","sum"),
                                                                                                                            avgRebounds=("reboundsTotal","mean"),
                                                                                                                            sdRebounds=("reboundsTotal","std"),
                                                                                                                            totalAssists=("assists","sum"),
                                                                                                                            avgAssists=("assists","mean"),
                                                                                                                            sdAssists=("assists","std"),
                                                                                                                            totalSteals=("steals","sum"),
                                                                                                                            avgSteals=("steals","mean"),
                                                                                                                            sdSteals=("steals","std"),
                                                                                                                            totalBlocks=("blocks","sum"),
                                                                                                                            avgBlocks=("blocks","mean"),
                                                                                                                            sdBlocks=("blocks","std"),
                                                                                                                            totalTurnovers=("turnovers","sum"),
                                                                                                                            avgTurnovers=("turnovers","mean"),
                                                                                                                            sdTurnovers=("turnovers","std"),
                                                                                                                            totalFouls=("foulsPersonal","sum"),
                                                                                                                            avgFouls=("foulsPersonal","mean"),
                                                                                                                            sdFouls=("foulsPersonal","std"),
                                                                                                                            totalPoints=("points","sum"),
                                                                                                                            avgPoints=("points","mean"),
                                                                                                                            sdPoints=("points","std"),
                                                                                                                            avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                                            totalPlusMinusPoints=("plusMinusPoints","sum"))

play_off_player_all_time_summary_stats["totalFieldGoalsPercentage"] = play_off_player_all_time_summary_stats["totalFieldGoalsMade"]/play_off_player_all_time_summary_stats["totalFieldGoalsAttempted"]
play_off_player_all_time_summary_stats["totalThreePointersPercentage"] \
    = play_off_player_all_time_summary_stats["totalThreePointersMade"]/play_off_player_all_time_summary_stats["totalThreePointersAttempted"]
play_off_player_all_time_summary_stats["totalFreeThrowsPercentage"] = play_off_player_all_time_summary_stats["totalFreeThrowsMade"]/play_off_player_all_time_summary_stats["totalFreeThrowsAttempted"]

In [47]:
regular_season_player_all_time_summary_stats

totalMinutesPlayed  \
personName         teamName                                     
A.J. Lawson        Dallas Mavericks                       387   
                   Minnesota Timberwolves                   1   
AJ Green           Milwaukee Bucks                        917   
AJ Griffin         Atlanta Hawks                         1529   
AJ Hammons         Dallas Mavericks                       154   
...                                                       ...   
Zoran Dragic       Miami Heat                              58   
                   Phoenix Suns                            10   
Zydrunas Ilgauskas Miami Heat                            1106   
Zylan Cheatham     New Orleans Pelicans                    48   
                   Utah Jazz                                5   

                                           avgMinutesPlayed  \
personName         teamName                                   
A.J. Lawson        Dallas Mavericks                6.910714   
                   Minnesota Timberwolves          1.000000   
AJ Green           Milwaukee Bucks                10.076923   
AJ Griffin         Atlanta Hawks                  16.619565   
AJ Hammons         Dallas Mavericks                7.000000   
...                                                     ...   
Zoran Dragic       Miami Heat                      5.800000   
                   Phoenix Suns                    1.666667   
Zydrunas Ilgauskas Miami Heat                     15.361111   
Zylan Cheatham     New Orleans Pelicans           12.000000   
                   Utah Jazz                       5.000000   

                                           totalFieldGoalsMade  \
personName         teamName                                      
A.J. Lawson        Dallas Mavericks                         75   
                   Minnesota Timberwolves                    1   
AJ Green           Milwaukee Bucks                         136   
AJ Griffin         Atlanta Hawks                           266   
AJ Hammons         Dallas Mavericks                         17   
...                                                        ...   
Zoran Dragic       Miami Heat                                9   
                   Phoenix Suns                              2   
Zydrunas Ilgauskas Miami Heat                              162   
Zylan Cheatham     New Orleans Pelicans                      6   
                   Utah Jazz                                 0   

                                           avgFieldGoalsMade  \
personName         teamName                                    
A.J. Lawson        Dallas Mavericks                 1.339286   
                   Minnesota Timberwolves           1.000000   
AJ Green           Milwaukee Bucks                  1.494505   
AJ Griffin         Atlanta Hawks                    2.891304   
AJ Hammons         Dallas Mavericks                 0.772727   
...                                                      ...   
Zoran Dragic       Miami Heat                       0.900000   
                   Phoenix Suns                     0.333333   
Zydrunas Ilgauskas Miami Heat                       2.250000   
Zylan Cheatham     New Orleans Pelicans             1.500000   
                   Utah Jazz                        0.000000   

                                           sdFieldGoalsMade  \
personName         teamName                                   
A.J. Lawson        Dallas Mavericks                1.575688   
                   Minnesota Timberwolves               NaN   
AJ Green           Milwaukee Bucks                 1.675799   
AJ Griffin         Atlanta Hawks                   2.341487   
AJ Hammons         Dallas Mavericks                1.109776   
...                                                     ...   
Zoran Dragic       Miami Heat                      2.846050   
                   Phoenix Suns                    0.516398   
Zydrunas Ilgauskas Miami Heat                      1.85

In [48]:

play_off_player_all_time_summary_stats

,,totalMinutesPlayed,avgMinutesPlayed,totalFieldGoalsMade,avgFieldGoalsMade,sdFieldGoalsMade,totalFieldGoalsAttempted,avgFieldGoalsAttempted,avgFieldGoalPercentage,totalThreePointersMade,avgThreePointersMade,...,avgFouls,sdFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints,totalFieldGoalsPercentage,totalThreePointersPercentage,totalFreeThrowsPercentage
personName,teamName,,,,,,,,,,,,,,,,,,,,,
A.J. Lawson,Dallas Mavericks,0,0.000000,0,0.000000,NaN,0,0.000000,0.000000,0,0.000000,...,0.000000,NaN,0,0.000000,NaN,-2.000000,-2,NaN,NaN,NaN
AJ Green,Milwaukee Bucks,70,10.000000,6,0.857143,1.214986,19,2.714286,0.275571,2,0.285714,...,1.428571,0.975900,17,2.428571,3.258688,-0.428571,-3,0.315789,0.142857,1.000000
AJ Lawson,Dallas Mavericks,26,2.600000,4,0.400000,0.516398,9,0.900000,0.250000,2,0.200000,...,0.100000,0.316228,11,1.100000,1.286684,-1.200000,-12,0.444444,0.333333,0.500000
AJ Price,Indiana Pacers,83,9.222222,14,1.555556,1.333333,37,4.111111,0.325556,7,0.777778,...,0.555556,0.726483,44,4.888889,4.833333,-2.666667,-24,0.378378,0.437500,0.900000
Aaron Brooks,Chicago Bulls,126,10.500000,21,1.750000,1.055290,61,5.083333,0.361750,8,0.666667,...,1.333333,1.302678,54,4.500000,3.370999,-3.166667,-38,0.344262,0.307692,0.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zeke Nnaji,Denver Nuggets,47,3.133333,9,0.600000,0.736788,16,1.066667,0.466667,5,0.333333,...,0.266667,0.457738,25,1.666667,1.988060,0.333333,5,0.562500,0.454545,0.500000
Zhaire Smith,Philadelphia 76ers,5,2.500000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,...,0.000000,0.000000,0,0.000000,0.000000,6.000000,12,NaN,NaN,NaN
Zhou Qi,Houston Rockets,4,1.333333,1,0.333333,0.577350,1,0.333333,0.333333,0,0.000000,...,0.000000,0.000000,2,0.666667,1.154701,0.000000,0,1.000000,NaN,NaN


By team, all time.

In [58]:
#Add total minutes all players played on a team per season

regular_season_player_all_time_stats = regular_season_player_summary_stats.groupby(["teamName"]).agg(totalMinutesPlayersPlayed=("totalMinutesPlayed","sum"))

play_off_player_all_time_stats = play_off_player_summary_stats.groupby(["teamName"]).agg(totalMinutesPlayersPlayed=("totalMinutesPlayed","sum"))

regular_season_team_all_time_summary_stats = regular_season_totals.groupby(["teamName"]).agg(totalWins=("WL",lambda total: total.str.count("W").sum()),
                                                                                               totalLosses=("WL",lambda total: total.str.count("L").sum()),
                                                                                               avgWins=("WL",lambda total: total.str.count("W").mean()),
                                                                                               avgLosses=("WL",lambda total: total.str.count("L").mean()),
                                                                                               totalMinutesPlayed=("minutes","sum"),
                                                                                               avgMinutesPlayed=("minutes","mean"),
                                                                                                totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                avgFieldGoalsPercentage=("fieldGoalsPercentage","mean"),
                                                                                                sdFieldGoalsPercentage=("fieldGoalsPercentage","std"),
                                                                                                totalThreePointersMade=("threePointersMade","sum"),
                                                                                                avgThreePointersMade=("threePointersMade","mean"),
                                                                                                sdThreePointersMade=("threePointersMade","std"),
                                                                                                totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                sdThreePointersPercentage=("threePointersPercentage","std"),
                                                                                                totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                sdFreeThrowsPercentage=("freeThrowsPercentage","std"),
                                                                                                totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                totalRebounds=("reboundsTotal","sum"),
                                                                                                avgRebounds=("reboundsTotal","mean"),
                                                                                                sdRebounds=("reboundsTotal","std"),
                                                                                                totalAssists=("assists","sum"),
                                                                                                avgAssists=("assists","mean"),
                                                                                                sdAssists=("assists","std"),
                                                                                                totalTurnovers=("turnovers","sum"),
                                                                                                avgTurnovers=("turnovers","mean"),
                                                                                                sdTurnovers=("turnovers","std"),
                                                                                                totalSteals=("steals","sum"),
                                                                                                avgSteals=("steals","mean"),
                                                                                                sdSteals=("steals","std"),
                                                                                                totalBlocks=("blocks","sum"),
                                                                                                avgBlocks=("blocks","mean"),
                                                                                                sdBlocks=("blocks","std"),
                                                                                                totalOpponentBlocks=("opponentBlocks","sum"),
                                                                                                avgOpponentBlocks=("opponentBlocks","mean"),
                                                                                                sdOpponentBlocks=("opponentBlocks","std"),
                                                                                                totalFouls=("foulsPersonal","sum"),
                                                                                                avgFouls=("foulsPersonal","mean"),
                                                                                                sdFouls=("foulsPersonal","std"),
                                                                                                totalDrawnFouls=("foulsPersonalDrawn","sum"),
                                                                                                avgDrawnFouls=("foulsPersonalDrawn","mean"),
                                                                                                sdDrawnFouls=("foulsPersonalDrawn","std"),
                                                                                                totalPoints=("points","sum"),
                                                                                                avgPoints=("points","mean"),
                                                                                                sdPoints=("points","std"),
                                                                                                avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                totalPlusMinusPoints=("plusMinusPoints","sum"))

regular_season_team_all_time_summary_stats["totalFieldGoalsPercentage"] \
    = regular_season_team_all_time_summary_stats["totalFieldGoalsMade"]/regular_season_team_all_time_summary_stats["totalFieldGoalsAttempted"]
regular_season_team_all_time_summary_stats["totalThreePointersPercentage"]\
    = regular_season_team_all_time_summary_stats["totalThreePointersMade"]/regular_season_team_all_time_summary_stats["totalThreePointersAttempted"]
regular_season_team_all_time_summary_stats["totalFreeThrowsPercentage"] \
    = regular_season_team_all_time_summary_stats["totalFreeThrowsMade"]/regular_season_team_all_time_summary_stats["totalFreeThrowsAttempted"]
regular_season_team_all_time_summary_stats = pd.merge(regular_season_team_all_time_summary_stats,regular_season_player_all_time_stats,how="left",on=["seasonYear","teamName"])

play_off_team_all_time_summary_stats = play_off_totals.groupby(["teamName"]).agg(totalWins=("WL",lambda total: total.str.count("W").sum()),
                                                                                               totalLosses=("WL",lambda total: total.str.count("L").sum()),
                                                                                               avgWins=("WL",lambda total: total.str.count("W").mean()),
                                                                                               avgLosses=("WL",lambda total: total.str.count("L").mean()),
                                                                                               totalMinutesPlayed=("minutes","sum"),
                                                                                               avgMinutesPlayed=("minutes","mean"),
                                                                                                totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                avgFieldGoalsPercentage=("fieldGoalsPercentage","mean"),
                                                                                                sdFieldGoalsPercentage=("fieldGoalsPercentage","std"),
                                                                                                totalThreePointersMade=("threePointersMade","sum"),
                                                                                                avgThreePointersMade=("threePointersMade","mean"),
                                                                                                sdThreePointersMade=("threePointersMade","std"),
                                                                                                totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                sdThreePointersPercentage=("threePointersPercentage","std"),
                                                                                                totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                sdFreeThrowsPercentage=("freeThrowsPercentage","std"),
                                                                                                totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                totalRebounds=("reboundsTotal","sum"),
                                                                                                avgRebounds=("reboundsTotal","mean"),
                                                                                                sdRebounds=("reboundsTotal","std"),
                                                                                                totalAssists=("assists","sum"),
                                                                                                avgAssists=("assists","mean"),
                                                                                                sdAssists=("assists","std"),
                                                                                                totalTurnovers=("turnovers","sum"),
                                                                                                avgTurnovers=("turnovers","mean"),
                                                                                                sdTurnovers=("turnovers","std"),
                                                                                                totalSteals=("steals","sum"),
                                                                                                avgSteals=("steals","mean"),
                                                                                                sdSteals=("steals","std"),
                                                                                                totalBlocks=("blocks","sum"),
                                                                                                avgBlocks=("blocks","mean"),
                                                                                                sdBlocks=("blocks","std"),
                                                                                                totalOpponentBlocks=("opponentBlocks","sum"),
                                                                                                avgOpponentBlocks=("opponentBlocks","mean"),
                                                                                                sdOpponentBlocks=("opponentBlocks","std"),
                                                                                                totalFouls=("foulsPersonal","sum"),
                                                                                                avgFouls=("foulsPersonal","mean"),
                                                                                                sdFouls=("foulsPersonal","std"),
                                                                                                totalDrawnFouls=("foulsPersonalDrawn","sum"),
                                                                                                avgDrawnFouls=("foulsPersonalDrawn","mean"),
                                                                                                sdDrawnFouls=("foulsPersonalDrawn","std"),
                                                                                                totalPoints=("points","sum"),
                                                                                                avgPoints=("points","mean"),
                                                                                                sdPoints=("points","std"),
                                                                                                avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                totalPlusMinusPoints=("plusMinusPoints","sum"))

play_off_team_all_time_summary_stats["totalFieldGoalsPercentage"] \
    = play_off_team_all_time_summary_stats["totalFieldGoalsMade"]/play_off_team_all_time_summary_stats["totalFieldGoalsAttempted"]
play_off_team_all_time_summary_stats["totalThreePointersPercentage"] \
    = play_off_team_all_time_summary_stats["totalThreePointersMade"]/play_off_team_all_time_summary_stats["totalThreePointersAttempted"]
play_off_team_all_time_summary_stats["totalFreeThrowsPercentage"] \
    = play_off_team_all_time_summary_stats["totalFreeThrowsMade"]/play_off_team_all_time_summary_stats["totalFreeThrowsAttempted"]
play_off_team_all_time_summary_stats = pd.merge(play_off_team_all_time_summary_stats,play_off_player_all_time_stats,how="left",on=["seasonYear","teamName"])

KeyError: 'seasonYear'

In [ ]:
regular_season_team_all_time_summary_stats

,totalWins,totalLosses,avgWins,avgLosses,totalMinutesPlayed,avgMinutesPlayed,totalFieldGoalsAttempted,avgFieldGoalsAttempted,totalFieldGoalsMade,avgFieldGoalsMade,...,avgDrawnFouls,sdDrawnFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints,totalFieldGoalsPercentage,totalThreePointersPercentage,totalFreeThrowsPercentage
teamName,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,551,556,0.497742,0.502258,53536.000,48.361337,94935,85.758808,43727,39.500452,...,19.894309,4.277093,117980,106.576332,14.195111,-0.460705,-510,0.460599,0.359501,0.776789
Boston Celtics,662,449,0.595860,0.404140,53763.000,48.391539,95241,85.725473,43888,39.503150,...,19.411341,4.194700,118282,106.464446,14.105725,3.594059,3993,0.460810,0.362620,0.788377
Brooklyn Nets,492,620,0.442446,0.557554,53816.000,48.395683,94399,84.891187,42986,38.656475,...,19.893885,4.319745,116822,105.055755,14.073631,-2.128597,-2367,0.455365,0.357237,0.766433
Charlotte Hornets,444,661,0.401810,0.598190,53430.000,48.352941,94312,85.350226,41875,37.895928,...,20.318552,4.098191,113611,102.815385,14.125548,-3.624434,-4005,0.444005,0.349805,0.762362
Chicago Bulls,565,540,0.511312,0.488688,53505.000,48.420814,94646,85.652489,43015,38.927602,...,18.997285,4.077042,114395,103.524887,13.187251,-0.087783,-97,0.454483,0.357981,0.778403
Cleveland Cavaliers,511,594,0.462443,0.537557,53390.000,48.316742,93549,84.659729,42696,38.638914,...,20.155656,4.059702,115356,104.394570,13.056870,-1.599095,-1767,0.456403,0.359682,0.756784
Dallas Mavericks,590,525,0.529148,0.470852,53940.000,48.376682,94777,85.001794,43617,39.118386,...,20.505830,4.406694,118554,106.326457,13.362007,1.048430,1169,0.460207,0.359460,0.772677
Denver Nuggets,635,478,0.570530,0.429470,53793.985,48.332421,96239,86.468104,45332,40.729560,...,20.556155,4.439718,121441,109.111411,12.948438,1.945193,2165,0.471036,0.359730,0.755947
Detroit Pistons,400,706,0.361664,0.638336,53468.000,48.343580,94878,85.784810,42426,38.359855,...,19.696203,4.228959,113451,102.577758,12.703005,-3.621157,-4005,0.447164,0.348187,0.732685


In [ ]:
play_off_team_all_time_summary_stats

,totalWins,totalLosses,avgWins,avgLosses,totalMinutesPlayed,avgMinutesPlayed,totalFieldGoalsAttempted,avgFieldGoalsAttempted,totalFieldGoalsMade,avgFieldGoalsMade,...,avgDrawnFouls,sdDrawnFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints,totalFieldGoalsPercentage,totalThreePointersPercentage,totalFreeThrowsPercentage
teamName,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,40,52,0.434783,0.565217,4436.0,48.217391,7656,83.217391,3312,36.000000,...,20.663043,4.274322,9008,97.913043,12.626200,-2.804348,-258,0.432602,0.333075,0.775013
Boston Celtics,98,79,0.553672,0.446328,8566.0,48.395480,14601,82.491525,6550,37.005650,...,20.180791,4.266821,18190,102.768362,13.115636,1.276836,226,0.448599,0.352503,0.798149
Brooklyn Nets,18,36,0.333333,0.666667,2617.0,48.462963,4463,82.648148,2011,37.240741,...,21.129630,5.005832,5556,102.888889,14.655370,-2.944444,-159,0.450594,0.353165,0.772763
Charlotte Hornets,3,8,0.272727,0.727273,528.0,48.000000,864,78.545455,358,32.545455,...,20.909091,4.548726,1000,90.909091,7.930379,-9.181818,-101,0.414352,0.330233,0.794776
Chicago Bulls,26,36,0.419355,0.580645,3011.0,48.564516,5076,81.870968,2155,34.758065,...,19.806452,3.784519,5746,92.677419,12.915582,-1.903226,-118,0.424547,0.328473,0.763636
Cleveland Cavaliers,63,37,0.630000,0.370000,4825.0,48.250000,8223,82.230000,3742,37.420000,...,21.270000,4.239914,10338,103.380000,12.864311,2.960000,296,0.455065,0.362347,0.745690
Dallas Mavericks,48,48,0.500000,0.500000,4618.0,48.104167,7856,81.833333,3575,37.239583,...,21.520833,4.026175,9949,103.635417,12.919280,-0.697917,-67,0.455066,0.368872,0.759031
Denver Nuggets,52,48,0.520000,0.480000,4840.0,48.400000,8617,86.170000,3968,39.680000,...,21.160000,4.189260,10759,107.590000,13.153403,-0.040000,-4,0.460485,0.356974,0.781416
Detroit Pistons,0,8,0.000000,1.000000,384.0,48.000000,688,86.000000,294,36.750000,...,20.125000,2.799872,772,96.500000,6.654751,-16.125000,-129,0.427326,0.349558,0.681818


By position, all time.

In [59]:
regular_season_position_all_time_summary_stats = regular_season_box_scores[~(regular_season_box_scores["comment"].str.contains("DND|DNP|NWT",na=False,case=False))]
regular_season_position_all_time_summary_stats = regular_season_position_all_time_summary_stats.groupby(["position"],dropna=False).agg(totalMinutesPlayed = ("minutes","sum"),
                                                                                                                                        avgMinutesPlayed=("minutes","mean"),
                                                                                                                                        totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                                                        avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                                                        sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                                                        totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                                                        avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                                                        avgFieldGoalPercentage=("fieldGoalsPercentage","mean"),
                                                                                                                                        totalThreePointersMade=("threePointersMade","sum"),
                                                                                                                                        avgThreePointersMade=("threePointersMade","mean"),
                                                                                                                                        sdThreePointersMade=("threePointersMade","std"),
                                                                                                                                        totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                                                        avgThreePointersAttempted=("threePointersAttempted","mean"),
                                                                                                                                        avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                                                        totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                                                        avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                                                        totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                                                        avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                                                        sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                                                        avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                                                        totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                                                        avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                                                        sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                                                        totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                                                        avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                                                        sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                                                        totalRebounds=("reboundsTotal","sum"),
                                                                                                                                        avgRebounds=("reboundsTotal","mean"),
                                                                                                                                        sdRebounds=("reboundsTotal","std"),
                                                                                                                                        totalAssists=("assists","std"),
                                                                                                                                        avgAssists=("assists","mean"),
                                                                                                                                        sdAssists=("assists","std"),
                                                                                                                                        totalSteals=("steals","sum"),
                                                                                                                                        avgSteals=("steals","mean"),
                                                                                                                                        sdSteals=("steals","std"),
                                                                                                                                        totalBlocks=("blocks","sum"),
                                                                                                                                        avgBlocks=("blocks","mean"),
                                                                                                                                        sdBlocks=("blocks","std"),
                                                                                                                                        totalTurnovers=("turnovers","sum"),
                                                                                                                                        avgTurnovers=("turnovers","mean"),
                                                                                                                                        sdTurnovers=("turnovers","std"),
                                                                                                                                        totalFouls=("foulsPersonal","sum"),
                                                                                                                                        avgFouls=("foulsPersonal","mean"),
                                                                                                                                        sdFouls=("foulsPersonal","std"),
                                                                                                                                        totalPoints=("points","sum"),
                                                                                                                                        avgPoints=("points","mean"),
                                                                                                                                        sdPoints=("points","std"),
                                                                                                                                        avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                                                        totalPlusMinusPoints=("plusMinusPoints","sum"))

regular_season_position_all_time_summary_stats["totalFieldGoalsPercentage"] = regular_season_position_all_time_summary_stats["totalFieldGoalsMade"]/ \
                                                                              regular_season_position_all_time_summary_stats["totalFieldGoalsAttempted"]
regular_season_position_all_time_summary_stats["totalThreePointersPercentage"] = regular_season_position_all_time_summary_stats["totalThreePointersMade"]/ \
                                                                                 regular_season_position_all_time_summary_stats["totalThreePointersAttempted"]
regular_season_position_all_time_summary_stats["totalFreeThrowsPercentage"] = regular_season_position_all_time_summary_stats["totalFreeThrowsMade"]/ \
                                                                              regular_season_position_all_time_summary_stats["totalFreeThrowsAttempted"]

play_off_position_all_time_summary_stats = play_off_box_scores[~play_off_box_scores["comment"].str.contains("DND|DNP|NWT",na=False,case=False)]
play_off_position_all_time_summary_stats = play_off_position_all_time_summary_stats.groupby(["position"],dropna=False).agg(totalMinutesPlayed = ("minutes","sum"),
                                                                                                                            avgMinutesPlayed=("minutes","mean"),
                                                                                                                            totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                                                            avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                                                            sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                                                            totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                                                            avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                                                            avgFieldGoalPercentage=("fieldGoalsPercentage","mean"),
                                                                                                                            totalThreePointersMade=("threePointersMade","sum"),
                                                                                                                            avgThreePointersMade=("threePointersMade","mean"),
                                                                                                                            sdThreePointersMade=("threePointersMade","std"),
                                                                                                                            totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                                                            avgThreePointersAttempted=("threePointersAttempted","mean"),
                                                                                                                            avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                                                            totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                                                            avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                                                            totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                                                            avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                                                            sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                                                            avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                                                            totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                                                            avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                                                            sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                                                            totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                                                            avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                                                            sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                                                            totalRebounds=("reboundsTotal","sum"),
                                                                                                                            avgRebounds=("reboundsTotal","mean"),
                                                                                                                            sdRebounds=("reboundsTotal","std"),
                                                                                                                            totalAssists=("assists","std"),
                                                                                                                            avgAssists=("assists","mean"),
                                                                                                                            sdAssists=("assists","std"),
                                                                                                                            totalSteals=("steals","sum"),
                                                                                                                            avgSteals=("steals","mean"),
                                                                                                                            sdSteals=("steals","std"),
                                                                                                                            totalBlocks=("blocks","sum"),
                                                                                                                            avgBlocks=("blocks","mean"),
                                                                                                                            sdBlocks=("blocks","std"),
                                                                                                                            totalTurnovers=("turnovers","sum"),
                                                                                                                            avgTurnovers=("turnovers","mean"),
                                                                                                                            sdTurnovers=("turnovers","std"),
                                                                                                                            totalFouls=("foulsPersonal","sum"),
                                                                                                                            avgFouls=("foulsPersonal","mean"),
                                                                                                                            sdFouls=("foulsPersonal","std"),
                                                                                                                            totalPoints=("points","sum"),
                                                                                                                            avgPoints=("points","mean"),
                                                                                                                            sdPoints=("points","std"),
                                                                                                                            avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                                                            totalPlusMinusPoints=("plusMinusPoints","sum"))

In [ ]:
regular_season_position_all_time_summary_stats

,totalMinutesPlayed,avgMinutesPlayed,totalFieldGoalsMade,avgFieldGoalsMade,sdFieldGoalsMade,totalFieldGoalsAttempted,avgFieldGoalsAttempted,avgFieldGoalPercentage,totalThreePointersMade,avgThreePointersMade,...,avgFouls,sdFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints,totalFieldGoalsPercentage,totalThreePointersPercentage,totalFreeThrowsPercentage
position,,,,,,,,,,,,,,,,,,,,,
C,912279,27.382609,166266,4.990575,3.076156,310802,9.328911,0.536364,11226,0.336955,...,2.698553,1.467335,417720,12.538120,7.710343,0.318706,10618,0.534958,0.346792,0.696139
F,1997666,29.980580,346576,5.201345,3.157402,743240,11.154400,0.455862,82149,1.232876,...,2.312808,1.450711,932157,13.989630,8.366858,0.338801,22575,0.466304,0.360781,0.775268
G,2077703,31.181760,372281,5.587120,3.214452,841458,12.628437,0.432528,115530,1.733852,...,2.163570,1.400480,1035005,15.533152,8.862222,0.367646,24497,0.442424,0.368486,0.816946
NaN,2891955,15.639713,426189,2.304833,2.171171,957936,5.180525,0.391191,119919,0.648523,...,1.509991,1.369040,1147931,6.208019,5.715058,-0.311988,-57690,0.444903,0.348438,0.743997


In [ ]:
play_off_position_all_time_summary_stats

,totalMinutesPlayed,avgMinutesPlayed,totalFieldGoalsMade,avgFieldGoalsMade,sdFieldGoalsMade,totalFieldGoalsAttempted,avgFieldGoalsAttempted,avgFieldGoalPercentage,totalThreePointersMade,avgThreePointersMade,...,avgTurnovers,sdTurnovers,totalFouls,avgFouls,sdFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints
position,,,,,,,,,,,,,,,,,,,,,
C,68378,28.949196,10673,4.518628,3.193046,20468,8.665538,0.510869,888,0.375953,...,1.537680,1.460354,6852,2.900931,1.469270,27607,11.687976,8.155494,0.124894,295
F,157062,33.247671,26478,5.604996,3.478829,57994,12.276461,0.442891,6272,1.327688,...,1.746613,1.619258,12717,2.691998,1.459660,72739,15.397756,9.419096,0.447290,2113
G,158991,33.656012,27853,5.896063,3.533991,64220,13.594412,0.421273,9045,1.914691,...,2.096105,1.707016,11916,2.522439,1.450330,78229,16.559907,9.663227,0.407917,1927
NaN,174355,13.317675,23541,1.798121,2.000447,53933,4.119539,0.353511,7280,0.556065,...,0.601436,0.886341,18385,1.404293,1.394553,64538,4.929575,5.355066,-0.331118,-4335


By win/loss, all time.

In [ ]:
regular_season_win_loss_all_time_summary_stats = regular_season_totals.groupby(["WL"]).agg(totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                            avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                            totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                            avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                            sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                            avgFieldGoalsPercentage=("fieldGoalsPercentage","mean"),
                                                                                            sdFieldGoalsPercentage=("fieldGoalsPercentage","std"),
                                                                                            totalThreePointersMade=("threePointersMade","sum"),
                                                                                            avgThreePointersMade=("threePointersMade","mean"),
                                                                                            sdThreePointersMade=("threePointersMade","std"),
                                                                                            totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                            avgThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                            avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                            sdThreePointersPercentage=("threePointersPercentage","std"),
                                                                                            totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                            avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                            sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                            totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                            avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                            avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                            sdFreeThrowsPercentage=("freeThrowsPercentage","std"),
                                                                                            totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                            avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                            sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                            totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                            avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                            sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                            totalRebounds=("reboundsTotal","sum"),
                                                                                            avgRebounds=("reboundsTotal","mean"),
                                                                                            sdRebounds=("reboundsTotal","std"),
                                                                                            totalAssists=("assists","sum"),
                                                                                            avgAssists=("assists","mean"),
                                                                                            sdAssists=("assists","std"),
                                                                                            totalTurnovers=("turnovers","sum"),
                                                                                            avgTurnovers=("turnovers","mean"),
                                                                                            sdTurnovers=("turnovers","std"),
                                                                                            totalSteals=("steals","sum"),
                                                                                            avgSteals=("steals","mean"),
                                                                                            sdSteals=("steals","std"),
                                                                                            totalBlocks=("blocks","sum"),
                                                                                            avgBlocks=("blocks","mean"),
                                                                                            sdBlocks=("blocks","std"),
                                                                                            totalOpponentBlocks=("opponentBlocks","sum"),
                                                                                            avgOpponentBlocks=("opponentBlocks","mean"),
                                                                                            sdOpponentBlocks=("opponentBlocks","std"),
                                                                                            totalFouls=("foulsPersonal","sum"),
                                                                                            avgFouls=("foulsPersonal","mean"),
                                                                                            sdFouls=("foulsPersonal","std"),
                                                                                            totalDrawnFouls=("foulsPersonalDrawn","sum"),
                                                                                            avgDrawnFouls=("foulsPersonalDrawn","mean"),
                                                                                            sdDrawnFouls=("foulsPersonalDrawn","std"),
                                                                                            totalPoints=("points","sum"),
                                                                                            avgPoints=("points","mean"),
                                                                                            sdPoints=("points","std"),
                                                                                            avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                            totalPlusMinusPoints=("plusMinusPoints","sum"))

regular_season_win_loss_all_time_summary_stats["totalFieldGoalsPercentage"] = regular_season_win_loss_all_time_summary_stats["totalFieldGoalsMade"]/ \
                                                                              regular_season_win_loss_all_time_summary_stats["totalFieldGoalsAttempted"]
regular_season_win_loss_all_time_summary_stats["totalThreePointersPercentage"] = regular_season_win_loss_all_time_summary_stats["totalThreePointersMade"]/ \
                                                                                 regular_season_win_loss_all_time_summary_stats["totalThreePointersAttempted"]
regular_season_win_loss_all_time_summary_stats["totalFreeThrowsPercentage"] = regular_season_win_loss_all_time_summary_stats["totalFreeThrowsMade"]/ \
                                                                              regular_season_win_loss_all_time_summary_stats["totalFreeThrowsAttempted"]

play_off_win_loss_all_time_summary_stats = play_off_totals.groupby(["WL"]).agg(totalFieldGoalsAttempted=("fieldGoalsAttempted","sum"),
                                                                                avgFieldGoalsAttempted=("fieldGoalsAttempted","mean"),
                                                                                totalFieldGoalsMade=("fieldGoalsMade","sum"),
                                                                                avgFieldGoalsMade=("fieldGoalsMade","mean"),
                                                                                sdFieldGoalsMade=("fieldGoalsMade","std"),
                                                                                avgFieldGoalsPercentage=("fieldGoalsPercentage","mean"),
                                                                                sdFieldGoalsPercentage=("fieldGoalsPercentage","std"),
                                                                                totalThreePointersMade=("threePointersMade","sum"),
                                                                                avgThreePointersMade=("threePointersMade","mean"),
                                                                                sdThreePointersMade=("threePointersMade","std"),
                                                                                totalThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                avgThreePointersAttempted=("threePointersAttempted","sum"),
                                                                                avgThreePointersPercentage=("threePointersPercentage","mean"),
                                                                                sdThreePointersPercentage=("threePointersPercentage","std"),
                                                                                totalFreeThrowsMade=("freeThrowsMade","sum"),
                                                                                avgFreeThrowsMade=("freeThrowsMade","mean"),
                                                                                sdFreeThrowsMade=("freeThrowsMade","std"),
                                                                                totalFreeThrowsAttempted=("freeThrowsAttempted","sum"),
                                                                                avgFreeThrowsAttempted=("freeThrowsAttempted","mean"),
                                                                                avgFreeThrowsPercentage=("freeThrowsPercentage","mean"),
                                                                                sdFreeThrowsPercentage=("freeThrowsPercentage","std"),
                                                                                totalReboundsOffensive=("reboundsOffensive","sum"),
                                                                                avgReboundsOffensive=("reboundsOffensive","mean"),
                                                                                sdReboundsOffensive=("reboundsOffensive","std"),
                                                                                totalReboundsDefensive=("reboundsDefensive","sum"),
                                                                                avgReboundsDefensive=("reboundsDefensive","mean"),
                                                                                sdReboundsDefensive=("reboundsDefensive","std"),
                                                                                totalRebounds=("reboundsTotal","sum"),
                                                                                avgRebounds=("reboundsTotal","mean"),
                                                                                sdRebounds=("reboundsTotal","std"),
                                                                                totalAssists=("assists","sum"),
                                                                                avgAssists=("assists","mean"),
                                                                                sdAssists=("assists","std"),
                                                                                totalTurnovers=("turnovers","sum"),
                                                                                avgTurnovers=("turnovers","mean"),
                                                                                sdTurnovers=("turnovers","std"),
                                                                                totalSteals=("steals","sum"),
                                                                                avgSteals=("steals","mean"),
                                                                                sdSteals=("steals","std"),
                                                                                totalBlocks=("blocks","sum"),
                                                                                avgBlocks=("blocks","mean"),
                                                                                sdBlocks=("blocks","std"),
                                                                                totalOpponentBlocks=("opponentBlocks","sum"),
                                                                                avgOpponentBlocks=("opponentBlocks","mean"),
                                                                                sdOpponentBlocks=("opponentBlocks","std"),
                                                                                totalFouls=("foulsPersonal","sum"),
                                                                                avgFouls=("foulsPersonal","mean"),
                                                                                sdFouls=("foulsPersonal","std"),
                                                                                totalDrawnFouls=("foulsPersonalDrawn","sum"),
                                                                                avgDrawnFouls=("foulsPersonalDrawn","mean"),
                                                                                sdDrawnFouls=("foulsPersonalDrawn","std"),
                                                                                totalPoints=("points","sum"),
                                                                                avgPoints=("points","mean"),
                                                                                sdPoints=("points","std"),
                                                                                avgPlusMinusPoints=("plusMinusPoints","mean"),
                                                                                totalPlusMinusPoints=("plusMinusPoints","sum"))
play_off_win_loss_all_time_summary_stats["totalFieldGoalsPercentage"] = play_off_win_loss_all_time_summary_stats["totalFieldGoalsMade"]/ \
                                                                        play_off_win_loss_all_time_summary_stats["totalFieldGoalsAttempted"]
play_off_win_loss_all_time_summary_stats["totalThreePointersPercentage"] = play_off_win_loss_all_time_summary_stats["totalThreePointersMade"]/ \
                                                                           play_off_win_loss_all_time_summary_stats["totalThreePointersAttempted"]
play_off_win_loss_all_time_summary_stats["totalFreeThrowsPercentage"] = play_off_win_loss_all_time_summary_stats["totalFreeThrowsMade"]/ \
                                                                        play_off_win_loss_all_time_summary_stats["totalFreeThrowsAttempted"]


In [ ]:
regular_season_win_loss_all_time_summary_stats

,totalFieldGoalsAttempted,avgFieldGoalsAttempted,totalFieldGoalsMade,avgFieldGoalsMade,sdFieldGoalsMade,avgFieldGoalsPercentage,sdFieldGoalsPercentage,totalThreePointersMade,avgThreePointersMade,sdThreePointersMade,...,avgDrawnFouls,sdDrawnFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints,totalFieldGoalsPercentage,totalThreePointersPercentage,totalFreeThrowsPercentage
WL,,,,,,,,,,,,,,,,,,,,,
L,1432130,85.972506,622694,37.381078,4.956295,0.435454,0.048556,149694,8.986313,3.909025,...,19.594249,4.113890,1671379,100.334914,12.324504,-11.409233,-190055,0.434803,0.327200,0.755713
W,1421306,85.322728,688618,41.338576,5.095112,0.485391,0.050731,179130,10.753392,4.400842,...,20.469684,4.427487,1861434,111.744147,12.337983,11.409233,190055,0.484497,0.389196,0.775990


In [ ]:
play_off_win_loss_all_time_summary_stats

,totalFieldGoalsAttempted,avgFieldGoalsAttempted,totalFieldGoalsMade,avgFieldGoalsMade,sdFieldGoalsMade,avgFieldGoalsPercentage,sdFieldGoalsPercentage,totalThreePointersMade,avgThreePointersMade,sdThreePointersMade,...,avgDrawnFouls,sdDrawnFouls,totalPoints,avgPoints,sdPoints,avgPlusMinusPoints,totalPlusMinusPoints,totalFieldGoalsPercentage,totalThreePointersPercentage,totalFreeThrowsPercentage
WL,,,,,,,,,,,,,,,,,,,,,
L,98300,83.234547,41798,35.392041,4.749084,0.426086,0.048997,10606,8.980525,3.801043,...,20.678239,3.996580,114496,96.948349,11.738747,-11.956816,-14121,0.425209,0.319776,0.756900
W,98315,83.247248,46747,39.582557,5.045488,0.476354,0.051971,12879,10.905165,4.435544,...,21.548688,4.328311,128617,108.905165,12.060489,11.956816,14121,0.475482,0.383749,0.780628


## Advanced Summary Statistics

Advanced metrics for player and team-level analysis

In [ ]:
#Team statistics to use to expand upon the player summary statistics

regular_season_team_stats = regular_season_team_summary_stats.rename(columns={"totalMinutesPlayersPlayed":"teamMinutesPlayed","totalFreeThrowsAttempted":"teamFreeThrowsAttempted","totalFieldGoalsAttempted":"teamFieldGoalsAttempted","totalTurnovers":"teamTurnovers"})

regular_season_team_stats = regular_season_team_stats[["teamMinutesPlayed","teamFreeThrowsAttempted","teamFieldGoalsAttempted","teamTurnovers"]]

play_off_team_stats = play_off_team_summary_stats.rename(columns={"totalMinutesPlayersPlayed":"teamMinutesPlayed","totalFreeThrowsAttempted":"teamFreeThrowsAttempted","totalFieldGoalsAttempted":"teamFieldGoalsAttempted","totalTurnovers":"teamTurnovers"})

play_off_team_stats = play_off_team_stats[["teamMinutesPlayed","teamFreeThrowsAttempted","teamFieldGoalsAttempted","teamTurnovers"]]

regular_season_team_all_time_stats = regular_season_team_all_time_summary_stats.rename(columns={"totalMinutesPlayersPlayed":"teamMinutesPlayed","totalFreeThrowsAttempted":"teamFreeThrowsAttempted","totalFieldGoalsAttempted":"teamFieldGoalsAttempted","totalTurnovers":"teamTurnovers"})

regular_season_team_all_time_stats = regular_season_team_all_time_stats[["teamMinutesPlayed","teamFreeThrowsAttempted","teamFieldGoalsAttempted","teamTurnovers"]]

play_off_team_all_time_stats = play_off_team_all_time_summary_stats.rename(columns={"totalMinutesPlayersPlayed":"teamMinutesPlayed","totalFreeThrowsAttempted":"teamFreeThrowsAttempted","totalFieldGoalsAttempted":"teamFieldGoalsAttempted","totalTurnovers":"teamTurnovers"})

play_off_team_all_time_stats = play_off_team_all_time_stats[["teamMinutesPlayed","teamFreeThrowsAttempted","teamFieldGoalsAttempted","teamTurnovers"]]


KeyError: "['teamMinutesPlayed'] not in index"

In [ ]:
regular_season_team_stats

totalWins  totalLosses   avgWins  avgLosses  \
teamName           seasonYear                                                
Atlanta Hawks      2010-11            44           38  0.536585   0.463415   
                   2011-12            40           26  0.606061   0.393939   
                   2012-13            44           38  0.536585   0.463415   
                   2013-14            38           44  0.463415   0.536585   
                   2014-15            60           22  0.731707   0.268293   
...                                  ...          ...       ...        ...   
Washington Wizards 2019-20            25           47  0.347222   0.652778   
                   2020-21            34           38  0.472222   0.527778   
                   2021-22            35           47  0.426829   0.573171   
                   2022-23            35           47  0.426829   0.573171   
                   2023-24            15           67  0.182927   0.817073   

                               totalMinutesPlayed  avgMinutesPlayed  \
teamName           seasonYear                                         
Atlanta Hawks      2010-11                 3946.0         48.121951   
                   2011-12                 3233.0         48.984848   
                   2012-13                 3971.0         48.426829   
                   2013-14                 3966.0         48.365854   
                   2014-15                 3946.0         48.121951   
...                                           ...               ...   
Washington Wizards 2019-20                 3471.0         48.208333   
                   2020-21                 3481.0         48.347222   
                   2021-22                 3966.0         48.365854   
                   2022-23                 3951.0         48.182927   
                   2023-24                 3946.0         48.121951   

                               teamFieldGoalsAttempted  \
teamName           seasonYear                            
Atlanta Hawks      2010-11                        6429   
                   2011-12                        5348   
                   2012-13                        6644   
                   2013-14                        6688   
                   2014-15                        6699   
...                                                ...   
Washington Wizards 2019-20                        6544   
                   2020-21                        6547   
                   2021-22                        7056   
                   2022-23                        7127   
                   2023-24                        7493   

                               avgFieldGoalsAttempted  totalFieldGoalsMade  \
teamName           seasonYear                                                
Atlanta Hawks      2010-11                  78.402439                 2971   
                   2011-12                  81.030303                 2429   
                   2012-13                  81.024390                 3084   
                   2013-14                  81.560976                 3061   
                   2014-15                  81.695122                 3121   
...                                               ...                  ...   
Washington Wizards 2019-20                  90.888889                 2990   
                   2020-21                  90.930556                 3108   
                   2021-22                  86.048780                 3327   
                   2022-23                  86.914634                 3456   
                   2023-24                  91.378049                 3523   

                               avgFieldGoalsMade  ...  avgDrawnFouls  \
teamName           seasonYear                     ...                  
Atlanta Hawks      2010-11             36.231707  ...      18.500000   
                   2011-12             36.803030  ...      19.136364   
                   2012-13        

In [ ]:
play_off_team_stats

teamMinutesPlayed  teamFreeThrowsAttempted  \
teamName           seasonYear                                               
Atlanta Hawks      2010-11                 576.0                      263   
                   2011-12                 293.0                       97   
                   2012-13                 288.0                      157   
                   2013-14                 336.0                      174   
                   2014-15                 778.0                      311   
...                                          ...                      ...   
Washington Wizards 2013-14                 533.0                      250   
                   2014-15                 485.0                      237   
                   2016-17                 629.0                      320   
                   2017-18                 288.0                      140   
                   2020-21                 240.0                      145   

                               teamFieldGoalsAttempted  teamTurnovers  
teamName           seasonYear                                          
Atlanta Hawks      2010-11                         910            149  
                   2011-12                         476             92  
                   2012-13                         464             73  
                   2013-14                         551             95  
                   2014-15                        1369            203  
...                                                ...            ...  
Washington Wizards 2013-14                         877            141  
                   2014-15                         845            145  
                   2016-17                        1139            179  
                   2017-18                         511             81  
                   2020-21                         443             67  

[224 rows x 4 columns]

In [ ]:
regular_season_team_all_time_stats

,teamMinutesPlayed,teamFreeThrowsAttempted,teamFieldGoalsAttempted,teamTurnovers
teamName,,,,
Atlanta Hawks,53536.000,24394,94935,16048
Boston Celtics,53763.000,23712,95241,15363
Brooklyn Nets,53816.000,25406,94399,16156
Charlotte Hornets,53430.000,25501,94312,14699
Chicago Bulls,53505.000,23827,94646,15430
Cleveland Cavaliers,53390.000,24875,93549,15707
Dallas Mavericks,53940.000,24331,94777,14456
Denver Nuggets,53793.985,26105,96239,16046
Detroit Pistons,53468.000,25180,94878,15684


In [ ]:
play_off_team_all_time_stats

,teamMinutesPlayed,teamFreeThrowsAttempted,teamFieldGoalsAttempted,teamTurnovers
teamName,,,,
Atlanta Hawks,4436.0,1969,7656,1235
Boston Celtics,8566.0,3889,14601,2375
Brooklyn Nets,2617.0,1263,4463,692
Charlotte Hornets,528.0,268,864,126
Chicago Bulls,3011.0,1320,5076,866
Cleveland Cavaliers,4825.0,2320,8223,1293
Dallas Mavericks,4618.0,2270,7856,1161
Denver Nuggets,4840.0,2260,8617,1213
Detroit Pistons,384.0,154,688,84
